# From Scratch

In [20]:
import torch 
import numpy as np 
import torchinfo 
from torchinfo import summary 
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [31]:
def equation(input:torch.Tensor): # z = 5 * a^2 + 3 * b^2
    return torch.matmul(torch.mul(input, input), torch.tensor([5, 3], dtype=torch.float32))

num_of_data = 100000
num_of_train_data = int(num_of_data * 0.7)
num_of_val_data =int(num_of_data * 0.3)
inputs = torch.randn(num_of_data, 2)
outputs = equation(inputs).view(-1,1)

batch_size = 500 
number_of_batches = num_of_data//batch_size

In [35]:
class DeepLearning():

    def __init__(self, n_inputs, n_hidden, n_outputs, lr = 0.01):
        self.lr = lr
        # Initilizaing empty layers         
        self.hidden_layer = torch.randn(n_hidden, n_inputs+1) # additional 1 is to account for the bias value
        self.output_layer = torch.randn(n_outputs, n_hidden+1) # additional 1 is to account for the bias value

    def activate(self, weights, inputs):
        return torch.matmul(inputs, torch.transpose(weights[:, :-1], 0, 1)) + weights[:, -1]

    def sigmoid(self, inputs):
        return 1.0 / (1.0 + np.exp(-inputs))

    def MSELoss(self, target, output):
        return torch.mean((target-output)**2)

    def forwardpropagate(self, inputs, targets):
        self.inputs = inputs
        self.target = targets
        out = self.activate(self.hidden_layer, self.inputs)
        out = self.sigmoid(out)
        self.hidden_nodes = out.clone()
        out = self.activate(self.output_layer, out)
        self.forward_res = out
        return self.MSELoss(self.target, out).item()

    def backpropagate(self): # BackPropagation       

        # Accounting for the output_layer, from 10 nodes to one single output node
        dc_dw2 = torch.mul(2 * (self.forward_res-self.target), self.hidden_nodes)
        dc_db2 = 2 * (self.forward_res-self.target)
        dc_dW2 = torch.cat((dc_dw2, dc_db2), dim = 1)
        dc_dW2 = torch.mean(dc_dW2, dim = 0).view(1,-1)

        # Accouting for the hidden_layer, from the 2 input nodes to the 10 hidden nodes
        sigmoid_derivative = torch.matmul(torch.transpose(self.hidden_nodes, 0, 1), (1-self.hidden_nodes))
        dc_dy = 2*(self.forward_res-self.target)
        dy_dhidden= self.output_layer[:, :-1]
        dx_dw1 = self.inputs
        dc_dW1 = torch.matmul(torch.transpose(torch.matmul(torch.matmul(dc_dy, dy_dhidden), sigmoid_derivative), 0, 1), dx_dw1)
        dc_dW1 = torch.cat((dc_dW1, torch.mean((torch.transpose(torch.matmul(torch.matmul(dc_dy, dy_dhidden), sigmoid_derivative), 0, 1)), dim =1).view(-1,1)), dim=1)

        # Optimizing
        self.output_layer -= self.lr * dc_dW2 
        self.hidden_layer -= self.lr * dc_dW1

In [42]:
X_train, y_train, X_val, y_val = inputs[:num_of_train_data], outputs[:num_of_train_data], inputs[num_of_train_data:], outputs[num_of_train_data:]
model = DeepLearning(2, 10, 1, lr = 0.001)

for epoch in range(10):
    # Training Loop - with backpropagation to adjust weights and biases 
    training_loss, count = 0, 0
    for batch_end_idx in range(batch_size, num_of_train_data, batch_size):
        train_input, train_output = X_train[batch_end_idx-batch_size:batch_end_idx], y_train[batch_end_idx-batch_size:batch_end_idx]
        training_loss += model.forwardpropagate(train_input, train_output)
        model.backpropagate()
        count += 1
    training_loss /= count 

    # Validation Loop - without backpropagation 
    validation_loss, count = 0, 0
    for batch_end_idx in range(batch_size, num_of_val_data, batch_size):
        val_input, val_output = X_val[batch_end_idx-batch_size:batch_end_idx], y_val[batch_end_idx-batch_size:batch_end_idx]
        validation_loss += model.forwardpropagate(val_input, val_output)
        count += 1
    validation_loss /= count 

    print(f"====Epoch {epoch} | Training Loss : {training_loss} | Validation Loss : {validation_loss} ====")

====Epoch 0 | Training Loss : 89.00325297623229 | Validation Loss : 82.54726649138887 ====
====Epoch 1 | Training Loss : 76.79070416457361 | Validation Loss : 75.14696987604691 ====
====Epoch 2 | Training Loss : 68.7238460238889 | Validation Loss : 68.23674366837841 ====
====Epoch 3 | Training Loss : 68.5056367778092 | Validation Loss : 68.21958729372186 ====
====Epoch 4 | Training Loss : 68.50751731378568 | Validation Loss : 68.21832042629435 ====
====Epoch 5 | Training Loss : 68.52732440207502 | Validation Loss : 68.22835211026467 ====
====Epoch 6 | Training Loss : 68.57223798902773 | Validation Loss : 68.21666167954267 ====
====Epoch 7 | Training Loss : 68.60415948552193 | Validation Loss : 68.23872582387116 ====
====Epoch 8 | Training Loss : 68.7059502087051 | Validation Loss : 68.28537814900027 ====
====Epoch 9 | Training Loss : 68.69425259048133 | Validation Loss : 68.54287719726562 ====
